<a href="https://colab.research.google.com/github/salim-hbk/ai-ml/blob/main/TPU_DistributedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import os
import random
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print("TensorFlow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

TensorFlow version 2.4.0


In [43]:
tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker']) 
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: grpc://10.23.92.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.23.92.146:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.23.92.146:8470']
Number of accelerators:  8


In [56]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.fashion_mnist.load_data()
train_x = tf.cast(tf.reshape(train_x, [-1, 28,28,1]), 'float32')/255.
test_x  = tf.cast(tf.reshape(test_x, [-1, 28,28,1]), 'float32')/255.

In [57]:
len(train_x)

60000

In [64]:
BATCH_SIZE = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE * strategy.num_replicas_in_sync
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(len(train_x)).batch(GLOBAL_BATCH_SIZE, drop_remainder=False)
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(GLOBAL_BATCH_SIZE, drop_remainder=False)


In [65]:
dist_train_data = strategy.experimental_distribute_dataset(train_data)
dist_test_data = strategy.experimental_distribute_dataset(test_data)

In [66]:
def base_model():
  input = tf.keras.layers.Input(shape=[28,28,1,])
  x = tf.keras.layers.Conv2D(32, 3, activation='relu')(input)
  x = tf.keras.layers.MaxPooling2D()(x)
  x = tf.keras.layers.Conv2D(64, 3, activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D()(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  output = tf.keras.layers.Dense(10, activation='softmax')(x)
  model = tf.keras.models.Model(inputs=input, outputs=output)
  optimizer = tf.keras.optimizers.Adam()
  return model

In [74]:
with strategy.scope():
  model = base_model()
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)

  def compute_loss(x, y, model):
    y_hat = model(x)
    persample_loss = loss_object(y, y_hat)
    return tf.nn.compute_average_loss(persample_loss, global_batch_size=GLOBAL_BATCH_SIZE), y_hat

  test_loss = tf.keras.metrics.Mean(name='test_loss')

  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
  test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
  optimizer = tf.keras.optimizers.Adam()

In [75]:
@tf.function
def distributed_training(dataset):
  loss = strategy.run(train_step, args=(dataset,))
  return strategy.reduce(tf.distribute.ReduceOp.SUM, loss, axis=None)

@tf.function
def distributed_testing(dataset):
  return strategy.run(test_step, args=(dataset,))


def train_step(dataset):
  x, y = dataset
  with tf.GradientTape() as tape:
    loss, y_hat = compute_loss(x, y, model)
  gradient = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradient, model.trainable_variables))
  train_accuracy.update_state(y, y_hat)
  return loss


def test_step(dataset):
  x, y = dataset
  y_hat = model(x)
  loss = loss_object(y, y_hat)
  test_loss.update_state(loss)
  test_accuracy.update_state(y, y_hat)

In [76]:
epochs = 5

with strategy.scope():
  for epoch in range(epochs):
    train_loss = 0.
    batch_num = 0

    for batch in dist_train_data:
      train_loss += distributed_training(batch)
      batch_num +=1
    train_loss= train_loss/ batch_num

    for batch in dist_test_data:
      distributed_testing(batch)

    template = ("Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}, Test Loss: {:.2f}, "
                  "Test Accuracy: {:.2f}")
    print (template.format(epoch+1, train_loss,
                            train_accuracy.result()*100, test_loss.result() / strategy.num_replicas_in_sync,
                            test_accuracy.result()*100))
    
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()



Epoch 1, Loss: 0.80, Accuracy: 72.02, Test Loss: 0.07, Test Accuracy: 77.83
Epoch 2, Loss: 0.46, Accuracy: 83.42, Test Loss: 0.05, Test Accuracy: 84.61
Epoch 3, Loss: 0.39, Accuracy: 86.10, Test Loss: 0.05, Test Accuracy: 84.11
Epoch 4, Loss: 0.36, Accuracy: 86.98, Test Loss: 0.05, Test Accuracy: 86.80
Epoch 5, Loss: 0.34, Accuracy: 87.92, Test Loss: 0.04, Test Accuracy: 87.17
